In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy.stats import spearmanr
from scipy.cluster import hierarchy

from typing import List

from xgboost import XGBClassifier

from catboost import CatBoostClassifier

from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import silhouette_score

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ARDRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier 

from sklearn.ensemble import RandomForestClassifier

from sklearn.decomposition import PCA

C:\Users\Mate\AppData\Roaming\Python\Python39\site-packages\statsmodels\compat\pandas.py:61: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [3]:
df = pd.read_pickle('../data/processed/train.pkl')
df

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
train_199995,0,11.4880,-0.4956,8.2622,3.5142,10.3404,11.6081,5.6709,15.1516,-0.6209,...,6.1415,13.2305,3.9901,0.9388,18.0249,-1.7939,2.1661,8.5326,16.6660,-17.8661
train_199996,0,4.9149,-2.4484,16.7052,6.6345,8.3096,-10.5628,5.8802,21.5940,-3.6797,...,4.9611,4.6549,0.6998,1.8341,22.2717,1.7337,-2.1651,6.7419,15.9054,0.3388
train_199997,0,11.2232,-5.0518,10.5127,5.6456,9.3410,-5.4086,4.5555,21.5571,0.1202,...,4.0651,5.4414,3.1032,4.8793,23.5311,-1.5736,1.2832,8.7155,13.8329,4.1995


In [57]:
cols_float = df.select_dtypes(include=["float64"]).columns.tolist() 
df[cols_float] = df[cols_float].apply(pd.to_numeric, downcast='float')
df.dtypes

target       int64
var_0      float32
var_1      float32
var_2      float32
var_3      float32
            ...   
var_195    float32
var_196    float32
var_197    float32
var_198    float32
var_199    float32
Length: 201, dtype: object

In [58]:
X=df.drop("target", axis=1)
y=df["target"]

In [59]:
skf = StratifiedKFold(n_splits=5,shuffle=False)
skf.get_n_splits(X, y)
print(skf)

StratifiedKFold(n_splits=5, random_state=None, shuffle=False)


In [60]:
pca = PCA(n_components=150)

In [61]:
#PCA 30 variables
pca_30=PCA(n_components=30)

#PCA 100 variables
pca_100=PCA(n_components=100)

#PCA 150 variables
pca_150=PCA(n_components=150)


In [62]:
lista_pca=[pca_30, pca_100, pca_150]

In [63]:
#Inicializo modelos LogisticRegression para primer test
lg = LogisticRegression(random_state=0,max_iter=5000)
lg_30 = LogisticRegression(random_state=0,max_iter=5000)
lg_100 = LogisticRegression(random_state=0,max_iter=5000)
lg_150 = LogisticRegression(random_state=0,max_iter=5000)

In [64]:
#==================================LogisticRegression================================================
results=[]
contador=0
lista=[]
for train_index, test_index in skf.split(X, y):
    X_train=X.iloc[train_index]
    X_test=X.iloc[test_index]
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    lg.fit(X_train,y_train)
    y_pred=np.asarray(lg.predict(X_test))
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')    
    recall = recall_score(y_test, y_pred,average='binary')
    f1 = f1_score(y_test, y_pred,average='binary')
    lista=[precision,recall]
    results.append(lista)
    print(cm,precision,recall,f1)
    contador+=1
    if contador==2:
        break

[[35450   531]
 [ 2951  1068]] 0.6679174484052532 0.26573774570788755 0.3802064791740833
[[35511   470]
 [ 2929  1090]] 0.6987179487179487 0.27121174421497884 0.39075103065065425


In [65]:
#==================================LogisticRegression PCA_30=============================================
contador=0
lista=[]
for train_index, test_index in skf.split(X, y):
    X_train=pca_30.fit_transform(X.iloc[train_index])
    X_test=pca_30.transform(X.iloc[test_index])
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    lg_30.fit(X_train, y_train)
    y_pred=np.asarray(lg_30.predict(X_test))
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')
    recall = recall_score(y_test, y_pred,average='binary')
    f1 = f1_score(y_test, y_pred,average='binary')
    print(cm,precision,recall,f1)
    lista=[precision,recall]
    results.append(lista)
    contador+=1
    if contador==2:
        break

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[35981     0]
 [ 4019     0]] 0.0 0.0 0.0
[[35981     0]
 [ 4018     1]] 1.0 0.0002488181139586962 0.0004975124378109454


In [66]:
#==================================LogisticRegression PCA_100=============================================
contador=0
lista=[]
for train_index, test_index in skf.split(X, y):
    X_train=pca_100.fit_transform(X.iloc[train_index])
    X_test=pca_100.transform(X.iloc[test_index])
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    lg_100.fit(X_train, y_train)
    y_pred=np.asarray(lg_100.predict(X_test))
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')
    recall = recall_score(y_test, y_pred,average='binary')
    f1 = f1_score(y_test, y_pred,average='binary')
    print(cm,precision,recall,f1)
    lista=[precision,recall]
    results.append(lista)
    contador+=1
    if contador==2:
        break

[[35778   203]
 [ 3663   356]] 0.6368515205724508 0.08857924856929585 0.15552643075578854
[[35809   172]
 [ 3717   302]] 0.6371308016877637 0.07514307041552624 0.13443133763632314


In [67]:
#==================================LogisticRegression PCA_150=============================================
contador=0
lista=[]
for train_index, test_index in skf.split(X, y):
    X_train=pca_150.fit_transform(X.iloc[train_index])
    X_test=pca_150.transform(X.iloc[test_index])
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    lg_150.fit(X_train, y_train)
    y_pred=np.asarray(lg_150.predict(X_test))
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')
    recall = recall_score(y_test, y_pred,average='binary')
    f1 = f1_score(y_test, y_pred,average='binary')
    print(cm,precision,recall,f1)
    lista=[precision,recall]
    results.append(lista)
    contador+=1
    if contador==2:
        break

[[35590   391]
 [ 3249   770]] 0.6632213608957795 0.19158994774819607 0.2972972972972973
[[35622   359]
 [ 3300   719]] 0.6669758812615956 0.17890022393630256 0.28212674122032566


In [68]:
#Inicializo modelos DecisionTreeClassifier para primer test
dtc = DecisionTreeClassifier(max_depth=3, random_state = 42)
dtc_30 = DecisionTreeClassifier(max_depth=3, random_state = 42)
dtc_100 = DecisionTreeClassifier(max_depth=3, random_state = 42)
dtc_150 = DecisionTreeClassifier(max_depth=3, random_state = 42)

In [69]:
#==================================DecisionTreeClassifier================================================
contador=0
lista=[]
for train_index, test_index in skf.split(X, y):
    X_train=X.iloc[train_index]
    X_test=X.iloc[test_index]
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    dtc.fit(X_train,y_train)
    y_pred=np.asarray(dtc.predict(X_test))
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')    
    recall = recall_score(y_test, y_pred,average='binary')
    f1 = f1_score(y_test, y_pred,average='binary')
    lista=[precision,recall]
    results.append(lista)
    print(cm,precision,recall,f1)
    contador+=1
    if contador==2:
        break

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[35981     0]
 [ 4019     0]] 0.0 0.0 0.0
[[35952    29]
 [ 3997    22]] 0.43137254901960786 0.005473998507091316 0.010810810810810811


In [70]:
#==================================DecisionTreeClassifier PCA_30=============================================
contador=0
lista=[]
for train_index, test_index in skf.split(X, y):
    X_train=pca_30.fit_transform(X.iloc[train_index])
    X_test=pca_30.transform(X.iloc[test_index])
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    dtc_30.fit(X_train, y_train)
    y_pred=np.asarray(dtc_30.predict(X_test))
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')
    recall = recall_score(y_test, y_pred,average='binary')
    f1 = f1_score(y_test, y_pred,average='binary')
    print(cm,precision,recall,f1)
    lista=[precision,recall]
    results.append(lista)
    contador+=1
    if contador==2:
        break

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[35981     0]
 [ 4019     0]] 0.0 0.0 0.0
[[35981     0]
 [ 4019     0]] 0.0 0.0 0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [71]:
#==================================DecisionTreeClassifier PCA_100=============================================
contador=0
lista=[]
for train_index, test_index in skf.split(X, y):
    X_train=pca_100.fit_transform(X.iloc[train_index])
    X_test=pca_100.transform(X.iloc[test_index])
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    dtc_100.fit(X_train, y_train)
    y_pred=np.asarray(dtc_100.predict(X_test))
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')
    recall = recall_score(y_test, y_pred,average='binary')
    f1 = f1_score(y_test, y_pred,average='binary')
    print(cm,precision,recall,f1)
    lista=[precision,recall]
    results.append(lista)
    contador+=1
    if contador==2:
        break

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[[35981     0]
 [ 4019     0]] 0.0 0.0 0.0
[[35981     0]
 [ 4019     0]] 0.0 0.0 0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [72]:
#==================================DecisionTreeClassifier PCA_150=============================================
contador=0
lista=[]
for train_index, test_index in skf.split(X, y):
    X_train=pca_150.fit_transform(X.iloc[train_index])
    X_test=pca_150.transform(X.iloc[test_index])
    y_train=y.iloc[train_index]
    y_test=y.iloc[test_index]
    dtc_150.fit(X_train, y_train)
    y_pred=np.asarray(dtc_150.predict(X_test))
    cm=confusion_matrix(y_test, y_pred)
    precision = precision_score(y_test, y_pred,average='binary')
    recall = recall_score(y_test, y_pred,average='binary')
    f1 = f1_score(y_test, y_pred,average='binary')
    print(cm,precision,recall,f1)
    lista=[precision,recall]
    results.append(lista)
    contador+=1
    if contador==2:
        break

[[35974     7]
 [ 4015     4]] 0.36363636363636365 0.0009952724558347848 0.0019851116625310174
[[35981     0]
 [ 4019     0]] 0.0 0.0 0.0


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [73]:
filas=["LogisticRegression_fulldata","LogisticRegression_fulldata",
       "LogisticRegression_pca30","LogisticRegression_pca30",
       "LogisticRegression_pca100","LogisticRegression_pca100",
       "LogisticRegression_pca150","LogisticRegression_pca150",
       "DecisionTreeClassifier_fulldata","DecisionTreeClassifier_fulldata",
       "DecisionTreeClassifier_pca30","DecisionTreeClassifier_pca30",
       "DecisionTreeClassifier_pca100","DecisionTreeClassifier_pca100",
       "DecisionTreeClassifier_pca150","DecisionTreeClassifier_pca150",
       ]
columnas=["PRECISION", "RECALL"]
len(filas)

16

In [76]:
df_results = pd.DataFrame(results, columns=columnas,index=filas)
df_results.sort_values(["RECALL","PRECISION"], ascending=False)


,PRECISION,RECALL
LogisticRegression_fulldata,0.698718,0.271212
LogisticRegression_fulldata,0.667917,0.265738
LogisticRegression_pca150,0.663221,0.191590
LogisticRegression_pca150,0.666976,0.178900
LogisticRegression_pca100,0.636852,0.088579
LogisticRegression_pca100,0.637131,0.075143
DecisionTreeClassifier_fulldata,0.431373,0.005474
DecisionTreeClassifier_pca150,0.363636,0.000995
LogisticRegression_pca30,1.000000,0.000249
LogisticRegression_pca30,0.000000,0.000000
